# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fabd018c550>
├── 'a' --> tensor([[-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291]])
└── 'x' --> <FastTreeValue 0x7fabd018cee0>
    └── 'c' --> tensor([[ 0.7481, -1.2031,  1.0124, -0.0324],
                        [ 1.2301,  0.7222,  1.1921, -0.0052],
                        [-1.1525, -1.1095,  0.3285, -0.2761]])

In [4]:
t.a

tensor([[-0.0981, -0.1857,  0.8352],
        [ 1.1619,  0.8450, -0.6291]])

In [5]:
%timeit t.a

62.3 ns ± 0.0264 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fabd018c550>
├── 'a' --> tensor([[-0.2325,  0.9501,  1.1828],
│                   [ 0.5647, -0.5311, -0.0254]])
└── 'x' --> <FastTreeValue 0x7fabd018cee0>
    └── 'c' --> tensor([[ 0.7481, -1.2031,  1.0124, -0.0324],
                        [ 1.2301,  0.7222,  1.1921, -0.0052],
                        [-1.1525, -1.1095,  0.3285, -0.2761]])

In [7]:
%timeit t.a = new_value

67 ns ± 0.0237 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291]]),
    x: Batch(
           c: tensor([[ 0.7481, -1.2031,  1.0124, -0.0324],
                      [ 1.2301,  0.7222,  1.1921, -0.0052],
                      [-1.1525, -1.1095,  0.3285, -0.2761]]),
       ),
)

In [10]:
b.a

tensor([[-0.0981, -0.1857,  0.8352],
        [ 1.1619,  0.8450, -0.6291]])

In [11]:
%timeit b.a

54 ns ± 0.19 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4951,  0.1077, -0.4042],
               [-1.1365, -0.3591, -0.8003]]),
    x: Batch(
           c: tensor([[ 0.7481, -1.2031,  1.0124, -0.0324],
                      [ 1.2301,  0.7222,  1.1921, -0.0052],
                      [-1.1525, -1.1095,  0.3285, -0.2761]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.179 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

903 ns ± 8.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 39.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fab2ca00fa0>
├── 'a' --> tensor([[[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]],
│           
│                   [[-0.0981, -0.1857,  0.8352],
│                    [ 1.1619,  0.8450, -0.6291]]])
└── 'x' --> <FastTreeValue 0x7fab2ca00250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 92.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fab25df2fd0>
├── 'a' --> tensor([[-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291],
│                   [-0.0981, -0.1857,  0.8352],
│                   [ 1.1619,  0.8450, -0.6291]])
└── 'x' --> <FastTreeValue 0x7fabd018c190>
    └── 'c' --> tensor([[ 0.7481, -1.2031,  1.0124, -0.0324],
                        [ 1.2301,  0.7222,  1.1921, -0.0052],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 98 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.4 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]],
       
               [[-0.0981, -0.1857,  0.8352],
                [ 1.1619,  0.8450, -0.6291]]]),
    x: Batch(
           c: tensor([[[ 0.7481, -1.2031,  1.0124, -0.0324],
                       [ 1.2301,  0.7222,  1.1921, -0.0052],
                       [-1.1525, -1.1095,  0.3285, -0.2761]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 63.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291],
               [-0.0981, -0.1857,  0.8352],
               [ 1.1619,  0.8450, -0.6291]]),
    x: Batch(
           c: tensor([[ 0.7481, -1.2031,  1.0124, -0.0324],
                      [ 1.2301,  0.7222,  1.1921, -0.0052],
                      [-1.1525, -1.1095,  0.3285, -0.2761],
                      [ 0.7481, -1.2031,  1.0124, -0.0324],
                      [ 1.2301,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 415 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 781 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
